In [2]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.3 MB/s 
     |████████████████████████████████| 3.4 MB 36.5 MB/s 
     |████████████████████████████████| 45 kB 2.0 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=8389c042bd88986711a9607729099e65aaddee80ce251873ba7ff32093ff6e58
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious


In [35]:
from pysat.formula import IDPool
from pysat.solvers import Cadical
from pysat.formula import CNF
cnf = CNF()
Atom = IDPool()

In [5]:
from google.colab import files
uploaded = files.upload()

Saving column_bin_10a_10p.csv to column_bin_10a_10p.csv


In [6]:
import pandas as pd
import io
 
arquivo = pd.read_csv(io.BytesIO(uploaded['column_bin_10a_10p.csv']))
arquivo

,LA <= 39.63,PI <= 48.12,PI <= 54.92,PI <= 63.52,PI <= 70.62,PI <= 80.61,PT <= 8.31,PT <= 12.36,PT <= 14.55,GS <= 37.89,P
0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,1
2,0,0,0,0,1,1,0,0,1,1,1
3,0,0,1,1,1,1,1,1,1,1,1
4,1,0,0,0,0,1,0,0,0,0,1
5,0,0,1,1,1,1,0,1,1,1,0
6,0,0,0,0,0,1,0,0,0,1,0
7,0,1,1,1,1,1,0,0,1,1,0
8,1,1,1,1,1,1,1,1,1,1,0
9,0,0,1,1,1,1,1,1,1,0,0


In [44]:
#Quantidade de pacientes do DataFrame
qtdPacientes=arquivo.shape[0]
print("Quantidade de Pacientes: ",qtdPacientes)

#Atributos do DataFrame
lista_atributos=arquivo.columns.tolist()
print("Atributos: ",lista_atributos)

#Quantidade de atributos do DataFrame
print("Quantidade de Atributos: ",arquivo.shape[1])

#Valoração
valoracao= [tuple(x) for x in arquivo.values]
print(valoracao)

#Quantidade de Regras
m=4

Quantidade de Pacientes:  10
Atributos:  ['LA <= 39.63', 'PI <= 48.12', 'PI <= 54.92', 'PI <= 63.52', 'PI <= 70.62', 'PI <= 80.61', 'PT <= 8.31', 'PT <= 12.36', 'PT <= 14.55', 'GS <= 37.89', 'P']
Quantidade de Atributos:  11
[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1), (0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1), (0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1), (1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1), (0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0), (0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0), (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0), (0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0)]


## **Restrição 1** 

In [48]:
def restricao1(m, lista_atributos):

    lista = []
    lista2 = []
    formula = []
    regra=0
    while True:
      if regra == m:
        break;
      for atributo in lista_atributos:
              if atributo != 'P':
                  lista=[]
                  lista.append(Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
                  lista2.append(lista)

                  lista=[]
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                  lista.append(Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
                  lista2.append(lista)

                  lista=[]
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                  lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                  lista.append(Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 's'))
                  lista2.append(lista)
      regra+=1

    return lista2

r1= restricao1(m, lista_atributos)
cnf.extend(r1)
print(r1)

[[1, -2, -3], [-1, 2, -3], [-1, -2, 3], [4, -5, -6], [-4, 5, -6], [-4, -5, 6], [7, -8, -9], [-7, 8, -9], [-7, -8, 9], [10, -11, -12], [-10, 11, -12], [-10, -11, 12], [13, -14, -15], [-13, 14, -15], [-13, -14, 15], [16, -17, -18], [-16, 17, -18], [-16, -17, 18], [19, -20, -21], [-19, 20, -21], [-19, -20, 21], [22, -23, -24], [-22, 23, -24], [-22, -23, 24], [25, -26, -27], [-25, 26, -27], [-25, -26, 27], [28, -29, -30], [-28, 29, -30], [-28, -29, 30], [31, -32, -33], [-31, 32, -33], [-31, -32, 33], [34, -35, -36], [-34, 35, -36], [-34, -35, 36], [37, -38, -39], [-37, 38, -39], [-37, -38, 39], [40, -41, -42], [-40, 41, -42], [-40, -41, 42], [43, -44, -45], [-43, 44, -45], [-43, -44, 45], [46, -47, -48], [-46, 47, -48], [-46, -47, 48], [49, -50, -51], [-49, 50, -51], [-49, -50, 51], [52, -53, -54], [-52, 53, -54], [-52, -53, 54], [55, -56, -57], [-55, 56, -57], [-55, -56, 57], [58, -59, -60], [-58, 59, -60], [-58, -59, 60], [61, -62, -63], [-61, 62, -63], [-61, -62, 63], [64, -65, -66], [-

## **Restrição 2** 

In [49]:
def restricao2(m, lista_atributos):
    lista1 = []
    regra=0
    while regra < m:
        lista2 = []
        for atributo in range(0, len(lista_atributos) - 1):
          if atributo !='P':
            lista2.append(-Atom.id(str(lista_atributos) + '-' + str(regra + 1) + '-' + 's'))
        lista1.append(lista2)
        regra+=1;
    return(lista1)

r2= restricao2(m, lista_atributos)
cnf.extend(r2)
print(r2)

[[-121, -121, -121, -121, -121, -121, -121, -121, -121, -121], [-122, -122, -122, -122, -122, -122, -122, -122, -122, -122], [-123, -123, -123, -123, -123, -123, -123, -123, -123, -123], [-124, -124, -124, -124, -124, -124, -124, -124, -124, -124]]


## **Restrição 3** 

In [59]:
def restricao3(m, lista_atributos, qtdPacientes, valoracao):
    lista2 = []
    j=0
    regra=0

    while True:
      if j == qtdPacientes:
          break
      if valoracao[j][-1] == 0: 
          while True:
            if regra == m:
              break;
            lista = []
            for a in lista_atributos:
                  if a != 'P':
                    if valoracao[j][lista_atributos.index(str(a))] == 1: 
                        lista.append(-Atom.id(str(lista_atributos) + '-' + str(regra + 1) + '-' + 'n')) 
                    elif valoracao[j][lista_atributos.index(str(a))] == 0:
                        lista.append(-Atom.id(str(lista_atributos) + '-' + str(regra + 1) + '-' + 'p'))
            regra+=1
            lista2.append(lista)
      j+=1
    return lista2

r3= restricao3(m, lista_atributos, qtdPacientes, valoracao)
cnf.extend(r3)
print(r3)

[[-125, -125, -121, -121, -121, -121, -125, -121, -121, -121], [-126, -126, -122, -122, -122, -122, -126, -122, -122, -122], [-127, -127, -123, -123, -123, -123, -127, -123, -123, -123], [-128, -128, -124, -124, -124, -124, -128, -124, -124, -124]]


## **Restrição 4** 

In [60]:
def restricao4(m, qtdPacientes, lista_atributos, valoracao):
    lista = []
    lista2 = []

    for j in range(qtdPacientes):
        if valoracao[j][-1] == 1:
            for regra in range(m):     
                for atributo in lista_atributos:
                    lista = []
                    if atributo != 'P':
                        if valoracao[j][lista_atributos.index(str(atributo))] == 1:
                            lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'n'))
                            lista.append(-Atom.id('C-' + str(regra+1) + '-'+str(j+1)))
                        else:
                            lista.append(-Atom.id(str(atributo) + '-' + str(regra + 1) + '-' + 'p'))
                            lista.append(-Atom.id('C-' + str(regra+1) + '-'+str(j+1)))
                        lista2.append(lista)

    return lista2

r4=restricao4(m ,qtdPacientes, lista_atributos, valoracao)
cnf.extend(r4)
print(r4)

[[-1, -129], [-4, -129], [-7, -129], [-10, -129], [-13, -129], [-16, -129], [-19, -129], [-22, -129], [-25, -129], [-28, -129], [-31, -130], [-34, -130], [-37, -130], [-40, -130], [-43, -130], [-46, -130], [-49, -130], [-52, -130], [-55, -130], [-58, -130], [-61, -131], [-64, -131], [-67, -131], [-70, -131], [-73, -131], [-76, -131], [-79, -131], [-82, -131], [-85, -131], [-88, -131], [-91, -132], [-94, -132], [-97, -132], [-100, -132], [-103, -132], [-106, -132], [-109, -132], [-112, -132], [-115, -132], [-118, -132], [-1, -133], [-4, -133], [-7, -133], [-10, -133], [-13, -133], [-17, -133], [-19, -133], [-22, -133], [-25, -133], [-28, -133], [-31, -134], [-34, -134], [-37, -134], [-40, -134], [-43, -134], [-47, -134], [-49, -134], [-52, -134], [-55, -134], [-58, -134], [-61, -135], [-64, -135], [-67, -135], [-70, -135], [-73, -135], [-77, -135], [-79, -135], [-82, -135], [-85, -135], [-88, -135], [-91, -136], [-94, -136], [-97, -136], [-100, -136], [-103, -136], [-107, -136], [-109, 

## **Restrição 5** 

In [62]:
def restricao5(m, qtdPacientes, valoracao, lista_atributos):
    lista2 = []
    for j in range(qtdPacientes):
        lista = []
        if valoracao[j][-1] == 1:
          for regra in range(m):
              lista.append(Atom.id('C-' + str(regra+1) + '-'+str(j+1)))
          lista2.append(lista)
    return lista2
r5= restricao5(m, qtdPacientes, valoracao, lista_atributos)
cnf.extend(r5)
print(r5)

[[129, 130, 131, 132], [133, 134, 135, 136], [137, 138, 139, 140], [141, 142, 143, 144], [145, 146, 147, 148]]


## **Teste** 

In [58]:
Formula = Cadical()
Formula.append_formula(cnf.clauses)

if Formula.solve():
    lista = Formula.get_model()
    print(lista)
    for atomID in lista:
      if atomID < 0:
        print("\nAtom.id:", atomID, '-', 'Not (', Atom.obj(atomID*-1), end = '), ')
      else:
        print("\nAtom.id:", atomID, '-', Atom.obj(atomID),  end = ', ')
else:
    print("Não foi achada uma valoração")


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, -91, -92, 93, -94, 95, -96, -97, -98, 99, -100, -101, 102, -103, -104, 105, -106, -107, 108, -109, -110, 111, -112, -113, 114, -115, -116, 117, -118, -119, 120, -121, -122, -123, -124, 125, 126, 127, 128, -129, -130, -131, 132, -133, -134, -135, 136, -137, -138, -139, 140, -141, -142, -143, 144, -145, -146, -147, 148]

Atom.id: 1 - LA <= 39.63-1-p, 
Atom.id: 2 - LA <= 39.63-1-n, 
Atom.id: 3 - LA <= 39.63-1-s, 
Atom.id: 4 - PI <= 48.12-1-p, 
Atom.id: 5 - PI <= 48.12-1-n, 
Atom.id: 6 - PI <= 48.12-1-s, 
Atom.id: 7 - PI <= 54.92-1-p, 
Atom.id: 8 - PI <= 54.92-1-n, 
Atom.id: 9 - PI <= 54.92-1-s, 
Atom.id: 10 - PI <= 63.52-1-p, 
Atom.id: 11 - PI